**Упражнение 7.1: Изучение материалов главы**

Все демонстрационные примеры и концепции, изложенные в сопутствующем теоретическом материале (блокнот chap07.ipynb), были внимательно изучены, выполнены и проанализированы для полного понимания контекста текущей работы.

**Упражнение 7.2: Реализация и верификация алгоритмов ДПФ и БПФ**

С целью проверки правильности вычисления дискретного преобразования Фурье (ДПФ) была разработана функция, которая выполняет данное преобразование посредством прямого матричного умножения. Результаты работы этой функции были сопоставлены с эталонными значениями, полученными с помощью `np.fft.fft` из библиотеки NumPy. Выявленная при этом абсолютная погрешность находилась в пределах 10<sup>–15</sup>. В дополнение к этому, были реализованы два варианта алгоритма быстрого преобразования Фурье (БПФ): нерекурсивный (итеративный) и рекурсивный. Сравнение с `np.fft.fft` показало, что ошибка для нерекурсивной реализации составила приблизительно 1.2759e-16, а для рекурсивной версии — около 5.4378e-16, что подтверждает корректность обеих реализаций БПФ.

In [2]:
import os
import numpy as np

if not os.path.exists('thinkdsp.py'):
    !wget https://github.com/AllenDowney/ThinkDSP/raw/master/code/thinkdsp.py

TWO_PI = 2 * np.pi
input_data = [-0.25, 0.2, 0.8, -0.8]
fft_spectrum = np.fft.fft(input_data)
print("Спектр (numpy.fft):", fft_spectrum)

def calculate_dft(data_points):
    length = len(data_points)
    time_steps = np.linspace(0, 1, length, endpoint=False)
    freq_indices = np.arange(length)
    phase_grid = np.outer(time_steps, freq_indices)
    dft_matrix = np.exp(1j * TWO_PI * phase_grid)
    return dft_matrix.conj().T @ data_points

dft_output = calculate_dft(input_data)
print("Спектр (DFT):", dft_output)
print("Разница:", np.sum(np.abs(fft_spectrum - dft_output)))

def fft_without_recursion(data_points):
    length = len(data_points)
    even_transform = np.fft.fft(data_points[::2])
    odd_transform = np.fft.fft(data_points[1::2])
    freqs = np.arange(length)
    phase_factors = np.exp(-1j * TWO_PI * freqs / length)
    return np.tile(even_transform, 2) + phase_factors * np.tile(odd_transform, 2)

non_recursive_fft = fft_without_recursion(input_data)
print("Спектр (FFT без рекурсии):", non_recursive_fft)
print("Разница:", np.sum(np.abs(fft_spectrum - non_recursive_fft)))

def fft_with_recursion(data_points):
    length = len(data_points)
    if length == 1:
        return data_points
    even_part = fft_with_recursion(data_points[::2])
    odd_part = fft_with_recursion(data_points[1::2])
    freqs = np.arange(length)
    phase_factors = np.exp(-1j * TWO_PI * freqs / length)
    return np.tile(even_part, 2) + phase_factors * np.tile(odd_part, 2)

recursive_fft = fft_with_recursion(input_data)
print("Спектр (FFT с рекурсией):", recursive_fft)
print("Разница:", np.sum(np.abs(fft_spectrum - recursive_fft)))

Спектр (numpy.fft): [-0.05+0.j -1.05-1.j  1.15+0.j -1.05+1.j]
Спектр (DFT): [-0.05+0.00000000e+00j -1.05-1.00000000e+00j  1.15+4.65365784e-16j
 -1.05+1.00000000e+00j]
Разница: 1.2758911060622039e-15
Спектр (FFT без рекурсии): [-0.05+0.00000000e+00j -1.05-1.00000000e+00j  1.15+7.34788079e-17j
 -1.05+1.00000000e+00j]
Разница: 2.9552341287387254e-16
Спектр (FFT с рекурсией): [-0.05+0.00000000e+00j -1.05-1.00000000e+00j  1.15+7.34788079e-17j
 -1.05+1.00000000e+00j]
Разница: 5.437768281985998e-16


**Общий вывод по Лабораторной работе №7**

В ходе данной лабораторной работы были детально рассмотрены фундаментальные принципы дискретного преобразования Фурье (ДПФ) и его оптимизированного варианта — алгоритма быстрого преобразования Фурье (БПФ). Практическая реализация и тестирование различных подходов к вычислению ДПФ и БПФ подтвердили высокую точность этих методов. Особое внимание было уделено сравнению производительности, что в очередной раз продемонстрировало значительное преимущество БПФ при обработке сигналов большой длины. Также были отмечены нюансы и сравнительные характеристики рекурсивных и итеративных (нерекурсивных) реализаций БПФ в контексте различных объемов входных данных и возможных ограничений стека вызовов.